In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
#read the data set 
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
#converting categorical data to numerical data
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
#Removing the unwanted cloumns from table
dataset=dataset.drop("User ID",axis=1)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [6]:
#input and output Split form data 
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [7]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [8]:
#standardize the value using StandardScaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
#model creation of KNeighborsClassifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {'n_neighbors':[7,10],'weights':['uniform','distance'],'algorithm':['auto','ball_tree','kd_tree','brute'],
              'metric':['minkowski'],'p':[2]} 
grid = GridSearchCV(KNeighborsClassifier(), param_grid,refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
# fitting the model for grid search 
grid.fit(X_train, y_train) 


Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'metric': ['minkowski'], 'n_neighbors': [7, 10],
                         'p': [2], 'weights': ['uniform', 'distance']},
             scoring='f1_weighted', verbose=3)

In [11]:
# print best parameter after tuning 
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print('the confusion metrix is \n',cm)
print('the classification report is \n',clf_report)

the confusion metrix is 
 [[78  7]
 [ 4 45]]
the classification report is 
               precision    recall  f1-score   support

           0       0.95      0.92      0.93        85
           1       0.87      0.92      0.89        49

    accuracy                           0.92       134
   macro avg       0.91      0.92      0.91       134
weighted avg       0.92      0.92      0.92       134



C:\Anconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [12]:
#Calculate the F1Score 
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'algorithm': 'auto', 'metric': 'minkowski', 'n_neighbors': 7, 'p': 2, 'weights': 'uniform'}: 0.9183922682195829


In [13]:
#calculate the roc_auc_score 
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9453781512605042

In [14]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_metric,param_n_neighbors,param_p,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.010459,0.008767,0.008359,0.010764,auto,minkowski,7,2,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.867478,0.886792,0.851527,0.944161,0.962264,0.902445,0.043310,1
1,0.003124,0.006249,0.003124,0.006249,auto,minkowski,7,2,distance,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.847141,0.906252,0.851527,0.944161,0.943041,0.898425,0.042363,5
2,0.000000,0.000000,0.003124,0.006249,auto,minkowski,10,2,uniform,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.847141,0.886792,0.849057,0.944161,0.962264,0.897883,0.047683,9
3,0.006253,0.007659,0.003121,0.006241,auto,minkowski,10,2,distance,"{'algorithm': 'auto', 'metric': 'minkowski', '...",0.847141,0.886792,0.851527,0.944161,0.943041,0.894533,0.042359,13
4,0.000000,0.000000,0.006250,0.007655,ball_tree,minkowski,7,2,uniform,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.867478,0.886792,0.851527,0.944161,0.962264,0.902445,0.043310,1
5,0.006252,0.007657,0.003126,0.006252,ball_tree,minkowski,7,2,distance,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.847141,0.906252,0.851527,0.944161,0.943041,0.898425,0.042363,5
6,0.003124,0.006249,0.006249,0.007653,ball_tree,minkowski,10,2,uniform,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.847141,0.886792,0.849057,0.944161,0.962264,0.897883,0.047683,9
7,0.000000,0.000000,0.003124,0.006249,ball_tree,minkowski,10,2,distance,"{'algorithm': 'ball_tree', 'metric': 'minkowsk...",0.847141,0.886792,0.851527,0.944161,0.943041,0.894533,0.042359,13
8,0.000000,0.000000,0.009385,0.007663,kd_tree,minkowski,7,2,uniform,"{'algorithm': 'kd_tree', 'metric': 'minkowski'...",0.867478,0.886792,0.851527,0.944161,0.962264,0.902445,0.043310,1
9,0.003121,0.006242,0.006242,0.007644,kd_tree,minkowski,7,2,distance,"{'algorithm': 'kd_tree', 'metric': 'minkowski'...",0.847141,0.906252,0.851527,0.944161,0.943041,0.898425,0.042363,5


In [15]:
Age_input=float(input("Age:"))
EstimatedSalary_input=float(input("EstimatedSalary:"))
Gender_input=float(input("Gender:"))


Age:25
EstimatedSalary:15000
Gender:0


In [17]:
Purchase_Prediction=grid.predict([[Age_input,EstimatedSalary_input,Gender_input]])
print("Purchase_Prediction={}".format(Purchase_Prediction))

Purchase_Prediction=[1]


C:\Anconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
